In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, date, time, timedelta
from functools import reduce

### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
%%time
ratings = pd.read_csv('../10-Pandas/ml-25m/ratings.csv')

ratings.head()

CPU times: user 7.04 s, sys: 19.9 s, total: 26.9 s
Wall time: 25.3 s


,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [5]:
%%time
tab1 = ratings.groupby(by='userId')['userId'].agg(cnt='count')
tab2 = tab1[tab1['cnt']>100]
tab3 = ratings.merge(tab2,how='inner',on='userId')[['userId','timestamp']]
tab4 = pd.pivot_table(tab3,values='timestamp',index=['userId'],aggfunc=(np.min,np.max))
tab4['diff']=tab4['amax']-tab4['amin']
print('{}'.format(timedelta(seconds=tab4['diff'].mean())))
# первый вариант


494 days, 4:25:49.007734
CPU times: user 8.44 s, sys: 47.9 s, total: 56.4 s
Wall time: 1min 12s


In [69]:
%%time
tab1 = ratings.groupby('userId').agg({'timestamp': ['min', 'max', 'count']})
tab2 = tab1[tab1['timestamp']['count']>100]
tab2.loc[:, ('timestamp','diff')] = tab2['timestamp']['max']-tab2['timestamp']['min']
print('{}'.format(timedelta(seconds=tab2.loc[:, ('timestamp','diff')].mean())))
# второй вариант


494 days, 4:25:49.007734
CPU times: user 1.28 s, sys: 1.54 s, total: 2.81 s
Wall time: 2.14 s


### Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
* аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [64]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
print(rzd)
print(auto)
print(air)
print(client_base)


   client_id  rzd_revenue
0        111         1093
1        112         2810
2        113        10283
3        114         5774
4        115          981
   client_id  auto_revenue
0        113         57483
1        114            83
2        115           912
3        116          4834
4        117            98
   client_id  air_revenue
0        115           81
1        116            4
2        117           13
3        118          173
   client_id          address
0        111  Комсомольская 4
1        112   Энтузиастов 8а
2        113  Левобережная 1а
3        114          Мира 14
4        115        ЗЖБИиДК 1
5        116    Строителей 18
6        117  Панфиловская 33
7        118     Мастеркова 4


In [65]:
df = [rzd, auto, air]
set1 = reduce(lambda  left,right: pd.merge(left,right,on=['client_id'],how='outer'), df).fillna(0)
print(set1)

   client_id  rzd_revenue  auto_revenue  air_revenue
0        111       1093.0           0.0          0.0
1        112       2810.0           0.0          0.0
2        113      10283.0       57483.0          0.0
3        114       5774.0          83.0          0.0
4        115        981.0         912.0         81.0
5        116          0.0        4834.0          4.0
6        117          0.0          98.0         13.0
7        118          0.0           0.0        173.0


In [66]:
set2 = set1.merge(client_base,on='client_id',how='left' )
print(set2)

   client_id  rzd_revenue  auto_revenue  air_revenue          address
0        111       1093.0           0.0          0.0  Комсомольская 4
1        112       2810.0           0.0          0.0   Энтузиастов 8а
2        113      10283.0       57483.0          0.0  Левобережная 1а
3        114       5774.0          83.0          0.0          Мира 14
4        115        981.0         912.0         81.0        ЗЖБИиДК 1
5        116          0.0        4834.0          4.0    Строителей 18
6        117          0.0          98.0         13.0  Панфиловская 33
7        118          0.0           0.0        173.0     Мастеркова 4


### Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
* У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
* Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
* Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

##### что-то я не понял кким образом от какого лица давать ответ. Напишу всё что я думаю по этой задаче.

    1) 
    Надеюсь, что у визитов и покупок, есть ни только дата, но и время. 
    
    Данные о местоположении так же содержат временную отметку, где была зафиксированна координата.
    
    Однако время покупки и время фиксации координаты могут не совподатать.
    Таким образом, нужно брать предшествующую коорднату и последующую, относительно времни покупки.
    
    Для того чтобы определить передшествующую и последующую координату, нужно датасет с последомательностью координав во времени, отсортировать по времени и пронумеровать строки, сделать так же и вов торой дата сет, только нумерацию сдлать уже как N-1, затем нужно соединить два датасета по номеру строки. Таким образом получим предшествующую и последующию координату,а также предшествуещее и последующее время (от и до) в обной строке. Важно чтобы дата/время "до" (предшествующей строки) ,была меньше и не равнялась дата/время "от" (последующей строки).
    
    После подготовки датасета с координатами и времени, нужно для каждого времени визита/покупки, найти интервал времени координат включающий время визита/покупки.

    2)
    Координаты до и после покупки могут находиться в значительном удалении друг от друга. Так как мы не можем более точно определитьего место положение во время покупки, то придется прибегнуть к каким-либо методам интерполяции. У нас есть три временные отметки и две точки на плоскости. Простейшим способом, будет нахождение координаты в момент покупки, к относительно пропорциональному сопоставлению времени покупк и между времени координаты "от" или "до".

    3) 
    После определения одной координаты покупки, мы можем сопоставить на карте области и регионы, где совершают больше всего визитов/покупкупок. А из этого можно сделать анализ по целевой группе клиентов. Так же возможна более обосновая маркентиговая стратегия. К примеру в каком райне стоит запустить более интентсивно рекламу.
    